In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/students-drop-out-prediction/train.csv
/kaggle/input/students-drop-out-prediction/test.csv


# **Importing and assessing data**

In [9]:
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
train_data = pd.read_csv("/kaggle/input/students-drop-out-prediction/train.csv")
test_data = pd.read_csv("/kaggle/input/students-drop-out-prediction/test.csv")

In [11]:
train_data.head()

,id,v_21,v_32,v_15,v_4,v_1,v_8,v_12,v_25,v_20,...,v_2,v_16,v_34,v_35,v_36,v_37,v_38,v_39,v_40,label
0,0,3,6,11,55,1,1,16,5,60,...,-0.3,0.79,1,93,47,1,48,26,0,1
1,1,8,94,12,65,1,4,22,4,173,...,5.4,6.51,6,29,37,25,53,35,7,2
2,2,3,166,10,155,1,1,16,1,309,...,0.6,2.02,1,8,34,4,43,38,2,1
3,3,3,6,13,121,1,1,16,1,6,...,0.6,2.02,7,37,40,27,43,65,0,1
4,4,4,51,18,21,1,17,22,118,168,...,8.4,12.51,16,22,25,25,24,23,1,1


In [12]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3796 entries, 0 to 3795
Data columns (total 43 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      3796 non-null   int64  
 1   v_21    3796 non-null   int64  
 2   v_32    3796 non-null   int64  
 3   v_15    3796 non-null   int64  
 4   v_4     3796 non-null   int64  
 5   v_1     3796 non-null   int64  
 6   v_8     3796 non-null   int64  
 7   v_12    3796 non-null   int64  
 8   v_25    3796 non-null   int64  
 9   v_20    3796 non-null   int64  
 10  v_18    3796 non-null   int64  
 11  v_3     3796 non-null   int64  
 12  v_11    3796 non-null   int64  
 13  v_14    3796 non-null   int64  
 14  v_26    3796 non-null   int64  
 15  v_27    3796 non-null   int64  
 16  v_30    3796 non-null   int64  
 17  v_9     3796 non-null   int64  
 18  v_31    3796 non-null   int64  
 19  v_28    3796 non-null   int64  
 20  v_13    3796 non-null   int64  
 21  v_33    3796 non-null   int64  
 22  

In [13]:
train_data.isna().sum()

id       0
v_21     0
v_32     0
v_15     0
v_4      0
v_1      0
v_8      0
v_12     0
v_25     0
v_20     0
v_18     0
v_3      0
v_11     0
v_14     0
v_26     0
v_27     0
v_30     0
v_9      0
v_31     0
v_28     0
v_13     0
v_33     0
v_17     0
v_19     0
v_29     0
v_23     0
v_24     0
v_10     0
v_5      0
v_6      0
v_22     0
v_0      0
v_7      0
v_2      0
v_16     0
v_34     0
v_35     0
v_36     0
v_37     0
v_38     0
v_39     0
v_40     0
label    0
dtype: int64

In [14]:
test_data.isna().sum()

id      0
v_21    0
v_32    0
v_15    0
v_4     0
v_1     0
v_8     0
v_12    0
v_25    0
v_20    0
v_18    0
v_3     0
v_11    0
v_14    0
v_26    0
v_27    0
v_30    0
v_9     0
v_31    0
v_28    0
v_13    0
v_33    0
v_17    0
v_19    0
v_29    0
v_23    0
v_24    0
v_10    0
v_5     0
v_6     0
v_22    0
v_0     0
v_7     0
v_2     0
v_16    0
v_34    0
v_35    0
v_36    0
v_37    0
v_38    0
v_39    0
v_40    0
dtype: int64

In [15]:
test_data.head()

,id,v_21,v_32,v_15,v_4,v_1,v_8,v_12,v_25,v_20,...,v_7,v_2,v_16,v_34,v_35,v_36,v_37,v_38,v_39,v_40
0,0,9,90,15,121,1,8,23,114,256,...,10.8,10.4,6.74,4,96,36,34,23,72,5
1,1,7,123,14,126,1,5,19,3,40,...,19.4,0.5,6.79,15,78,12,25,27,67,12
2,2,3,6,10,102,1,1,16,181,168,...,7.6,2.6,0.32,4,81,47,30,54,23,4
3,3,3,6,10,148,1,1,16,110,278,...,11.1,0.6,2.02,4,100,6,23,12,28,1
4,4,3,6,10,148,1,1,16,1,168,...,7.6,2.6,0.32,7,97,25,30,18,52,0


In [16]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1628 entries, 0 to 1627
Data columns (total 42 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      1628 non-null   int64  
 1   v_21    1628 non-null   int64  
 2   v_32    1628 non-null   int64  
 3   v_15    1628 non-null   int64  
 4   v_4     1628 non-null   int64  
 5   v_1     1628 non-null   int64  
 6   v_8     1628 non-null   int64  
 7   v_12    1628 non-null   int64  
 8   v_25    1628 non-null   int64  
 9   v_20    1628 non-null   int64  
 10  v_18    1628 non-null   int64  
 11  v_3     1628 non-null   int64  
 12  v_11    1628 non-null   int64  
 13  v_14    1628 non-null   int64  
 14  v_26    1628 non-null   int64  
 15  v_27    1628 non-null   int64  
 16  v_30    1628 non-null   int64  
 17  v_9     1628 non-null   int64  
 18  v_31    1628 non-null   int64  
 19  v_28    1628 non-null   int64  
 20  v_13    1628 non-null   int64  
 21  v_33    1628 non-null   int64  
 22  

# **Data Preprocessing**

In [17]:
y_train = train_data['label']

In [18]:
X_train = train_data.drop(columns = ['id','label'])

In [19]:
y_train.head()

0    1
1    2
2    1
3    1
4    1
Name: label, dtype: int64

In [20]:
y_train.shape

(3796,)

In [21]:
X_train.head()

,v_21,v_32,v_15,v_4,v_1,v_8,v_12,v_25,v_20,v_18,...,v_7,v_2,v_16,v_34,v_35,v_36,v_37,v_38,v_39,v_40
0,3,6,11,55,1,1,16,5,60,30,...,13.9,-0.3,0.79,1,93,47,1,48,26,0
1,8,94,12,65,1,4,22,4,173,199,...,14.9,5.4,6.51,6,29,37,25,53,35,7
2,3,166,10,155,1,1,16,1,309,30,...,11.1,0.6,2.02,1,8,34,4,43,38,2
3,3,6,13,121,1,1,16,1,6,23,...,11.1,0.6,2.02,7,37,40,27,43,65,0
4,4,51,18,21,1,17,22,118,168,106,...,10.9,8.4,12.51,16,22,25,25,24,23,1


In [22]:
X_train.shape

(3796, 41)

In [23]:
X_test = test_data.drop(columns = 'id')

In [24]:
X_test.head()

,v_21,v_32,v_15,v_4,v_1,v_8,v_12,v_25,v_20,v_18,...,v_7,v_2,v_16,v_34,v_35,v_36,v_37,v_38,v_39,v_40
0,9,90,15,121,1,8,23,114,256,108,...,10.8,10.4,6.74,4,96,36,34,23,72,5
1,7,123,14,126,1,5,19,3,40,261,...,19.4,0.5,6.79,15,78,12,25,27,67,12
2,3,6,10,102,1,1,16,181,168,103,...,7.6,2.6,0.32,4,81,47,30,54,23,4
3,3,6,10,148,1,1,16,110,278,103,...,11.1,0.6,2.02,4,100,6,23,12,28,1
4,3,6,10,148,1,1,16,1,168,12,...,7.6,2.6,0.32,7,97,25,30,18,52,0


In [25]:
X_test.shape

(1628, 41)

In [26]:
from sklearn.preprocessing import StandardScaler

In [27]:
scaler = StandardScaler()

In [28]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [29]:
X_train

array([[-0.29474947, -1.15547613, -0.2364351 , ...,  0.93711624,
        -0.63260337, -1.25223212],
       [ 0.51614268,  0.41306917,  0.15176025, ...,  1.2172975 ,
        -0.2319534 ,  1.42104754],
       [-0.29474947,  1.69642442, -0.62463045, ...,  0.65693499,
        -0.09840341, -0.48843793],
       ...,
       [ 0.02960739,  1.73207318,  0.92815094, ..., -1.08018878,
         1.72677981,  1.03915044],
       [-0.13257104,  0.32394728,  2.86912768, ...,  0.93711624,
         1.59322982,  1.03915044],
       [-0.29474947, -1.15547613, -0.62463045, ...,  0.60089874,
         0.25772991,  0.65725335]])

In [30]:
X_test

array([[ 0.64638235,  0.33455386,  1.46814858, ..., -0.40397071,
         1.39112375,  0.65386136],
       [ 0.33155499,  0.9186234 ,  1.05276   , ..., -0.18082564,
         1.17154362,  3.26198321],
       [-0.29809974, -1.15216859, -0.60879432, ...,  1.32540362,
        -0.76076153,  0.28127253],
       ...,
       [-0.29809974,  0.79472986, -0.60879432, ...,  0.65596839,
        -0.32160127, -1.20908281],
       [-0.29809974, -1.15216859, -0.60879432, ...,  1.38118989,
        -1.50733397, -0.46390514],
       [-0.29809974,  0.36995202, -0.60879432, ..., -1.18497848,
        -1.46341794, -0.09131631]])

# **Training Models**

In [31]:
from sklearn.model_selection import GridSearchCV

In [32]:
from sklearn.model_selection import RandomizedSearchCV

> **Baseline model**

In [33]:
from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier()
dummy_clf.fit(X_train,y_train)
dummy_clf.score(X_train,y_train)

0.49077976817702845

> **Logistic Regression**

In [34]:
from sklearn.linear_model import LogisticRegression
np.random.seed(42)
logreg_clf = LogisticRegression(max_iter = 1000,multi_class = "multinomial")
logreg_clf.fit(X_train,y_train)
logreg_clf.score(X_train,y_train)

0.743940990516333

In [35]:
y_pred_logreg = logreg_clf.predict(X_test)

> **Support Vector Classifier**

In [36]:
from sklearn.svm import SVC
np.random.seed(42)
svc_clf = SVC()
svc_clf.fit(X_train,y_train)
svc_clf.score(X_train,y_train)

0.8314014752370916

In [37]:
y_pred_svc = svc_clf.predict(X_test)

* **Support Vector Classifier CV**

In [38]:
from sklearn.svm import SVC
np.random.seed(42)
g=GridSearchCV(SVC(),{'kernel':['linear','rbf'],'C':[0.1,0.5,1]},cv=10)
g.fit(X_train,y_train)
g.best_params_

{'C': 1, 'kernel': 'rbf'}

In [39]:
np.random.seed(42)
svc_clf_cv = SVC(C = 35, kernel = 'rbf')
svc_clf_cv.fit(X_train,y_train)
svc_clf_cv.score(X_train,y_train)

0.9897260273972602

In [40]:
y_pred_svc_cv = svc_clf.predict(X_test)

> **K Nearest Neighbors**

In [41]:
from sklearn.neighbors import KNeighborsClassifier
np.random.seed(42)
kn_clf = KNeighborsClassifier()
kn_clf.fit(X_train,y_train)
kn_clf.score(X_train,y_train)

0.7597471022128557

> **Decision Tree Classifier**

In [42]:
from sklearn.tree import DecisionTreeClassifier
dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train,y_train)
dt_clf.score(X_train,y_train)

1.0

In [43]:
y_pred_dt = dt_clf.predict(X_test)

> **Gradient Boost Classifier**

In [44]:
from sklearn.ensemble import GradientBoostingClassifier

In [65]:
GB_clf = GradientBoostingClassifier(max_depth=8, 
                                   min_samples_leaf=7,
                                   min_samples_split=10, 
                                   random_state=1, 
                                   max_features="auto", 
                                   learning_rate=0.115, 
                                   warm_start=True)
GB_clf.fit(X_train,y_train)
GB_clf.score(X_train,y_train)

1.0

In [66]:
y_pred_gb = GB_clf.predict(X_test)

> **Random Forest Classifier**

In [67]:
from sklearn.ensemble import RandomForestClassifier
np.random.seed(42)
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train,y_train)
rf_clf.score(X_train,y_train)

1.0

* **Random Forest CV**

In [ ]:
rf_params = {'n_estimators':[10,20,50],
             'max_features':['auto','sqrt'],
             'max_depth':[2,4],
             'min_samples_split':[2,5],
             'min_samples_leaf':[1,2],
             'bootstrap':[True,False]}

In [ ]:
rf_clf_cv = RandomizedSearchCV(rf_clf,rf_params,cv = 3)
rf_clf_cv.fit(X_train,y_train)
print(rf_clf_cv.best_params_)

In [ ]:
rf_cv = RandomForestClassifier(n_estimators = 20, 
                             min_samples_split = 5, 
                             min_samples_leaf = 1, 
                             max_features = 'auto', 
                             max_depth = 4, 
                             bootstrap = False)
rf_cv.fit(X_train,y_train)
y_pred_rf = rf_clf.predict(X_test)

> **XG Boost Classifier**

In [47]:
from xgboost import XGBClassifier
xg_clf = XGBClassifier()
xg_clf.fit(X_train,y_train)
xg_clf.score(X_train,y_train)

1.0

In [ ]:
y_pred_xg = xg_clf.predict(X_test)

* **XG Boost CV**

In [61]:
xg_clf_cv = XGBClassifier(n_estimators = 1000)
xg_clf_cv.fit(X_train,y_train)
xg_clf_cv.score(X_train,y_train)

1.0

In [63]:
y_pred_xg_cv = xg_clf_cv.predict(X_test)

# **Creating CSV File**

In [64]:
test_data['label'] = y_pred_xg_cv
c1 = test_data['id'].to_numpy()
c2 = test_data['label'].to_numpy()
df=np.column_stack((c1,c2))
pd.DataFrame(df, columns=['id','label']).to_csv('subxg11.csv',index=False)